In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install json_repair

In [3]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
import json_repair

In [4]:
class Generator:
    def __init__(self, model, tokenizer):
        self.model, self.tokenizer = model, tokenizer
        self.mask = None

    def generate(self, messages: list, max_new_tokens: int = 2000, temperature: float = 0.1):
        def logits_processor(token_ids, logits):
            if self.mask is None:
                token_ids = torch.arange(logits.size(-1))
                decoded_tokens = self.tokenizer.batch_decode(token_ids.unsqueeze(1), skip_special_tokens=True)
                self.mask = torch.tensor([
                    any(0x4E00 <= ord(c) <= 0x9FFF or 0x3400 <= ord(c) <= 0x4DBF or 0xF900 <= ord(c) <= 0xFAFF for c in token)
                    for token in decoded_tokens
                ])
            logits[:, self.mask] = -float("inf")
            return logits

        text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            logits_processor=[logits_processor]
        )
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response

In [ ]:
base_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
adapter_path = "/gdrive/MyDrive/FineTuning_fromScratch/models"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=None)
model.load_adapter(adapter_path)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
llm = Generator(model, tokenizer)

In [6]:
def process_input(text, task):
    if task == "Extraction Details":
        messages = [
            {
                "role": "system",
                "content": "\n".join([
                    "You are an NLP data parser.",
                    "You will be provided by an Arabic text associated with a Pydantic scheme.",
                    "Generate the output in the same story language.",
                    "You have to extract JSON details from text according the Pydantic details.",
                    "Extract details as mentioned in text.",
                    "Do not generate any introduction or conclusion."
                ])
            },
            {
                "role": "user",
                "content": "\n".join([
                    "## Story:",
                    text.strip(),
                    "",
                    "## Pydantic Details:",
                    json.dumps({
                        "type": "object",
                        "properties": {
                            "story_title": {"type": "string", "minLength": 5, "maxLength": 300},
                            "story_keywords": {"type": "array", "items": {"type": "string"}, "minItems": 1},
                            "story_summary": {"type": "array", "items": {"type": "string"}, "minItems": 1, "maxItems": 5},
                            "story_category": {"type": "string", "enum": ["politics", "sports", "art", "technology", "economy", "health", "entertainment", "science", "not_specified"]},
                            "story_entities": {"type": "array", "items": {
                                "type": "object",
                                "properties": {
                                    "entity_value": {"type": "string"},
                                    "entity_type": {"type": "string", "enum": ["person-male", "person-female", "location", "organization", "event", "time", "quantity", "money", "product", "law", "disease", "artifact", "not_specified"]}
                                },
                                "required": ["entity_value", "entity_type"]
                            }, "minItems": 1, "maxItems": 10}
                        },
                        "required": ["story_title", "story_keywords", "story_summary", "story_category", "story_entities"]
                    }, ensure_ascii=False),
                    "",
                    "## Story Details:",
                    "```json"
                ])
            }
        ]
    elif task == "Translation":
        messages = [
            {
                "role": "system",
                "content": "\n".join([
                    "You are a professional translator.",
                    "You will be provided by an Arabic text.",
                    "You have to translate the text into the `Targeted Language`.",
                    "Follow the provided Scheme to generate a JSON",
                    "Do not generate any introduction or conclusion."
                ])
            },
            {
                "role": "user",
                "content": "\n".join([
                    "## Story:",
                    text.strip(),
                    "",
                    "## Pydantic Details:",
                    json.dumps({
                        "type": "object",
                        "properties": {
                            "translated_title": {"type": "string", "minLength": 5, "maxLength": 300},
                            "translated_content": {"type": "string", "minLength": 5}
                        },
                        "required": ["translated_title", "translated_content"]
                    }, ensure_ascii=False),
                    "",
                    "## Targeted Language:",
                    "English",
                    "",
                    "## Translated Story:",
                    "```json"
                ])
            }
        ]
    else:
        return "Chosse your mission: Translation or Extraction Details"

    response = llm.generate(messages)
    try:
        return json.dumps(json_repair.loads(response), ensure_ascii=False, indent=2)
    except:
        return response



In [7]:
iface = gr.Interface(
    fn=process_input,
    inputs=[
        gr.Textbox(label="Text (Arabic)", lines=10),
        gr.Dropdown(choices=["Extraction Details", "Translation"], label="Task")
    ],
    outputs=gr.Textbox(label="Result"),
    title="News Analyst",
    description="Enter text in Arabic and choose a task (extract details or translation) to get the result."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5d517a078cce906801.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
